# Desafio Bootcamp Ciência de Dados da DIO (ETL com Python)

**Contexto:** Se manter por dentro das notícias do clima na sua cidade é importante para planejar melhor o seu dia. O objetivo deste script foi usar o poder da IA Generativa para criar mensagens de sobre previsão do clima personalizadas que serão entregues de acordo com a cidade.

**Condições do Projeto:**

1. Foi consumido o API `https://openweathermap.org/` (API da OpenWeather) para obter os dados climáticos de cada cidade.
2. A requisição da API OpenWeather foi realizado dentro da estrutura de repetição a cada 60 segundos (1 minuto) para que não haja risco de exceder o limite de requisições por minuto na API.
3. Depois de obter os dados do clima da cidade, foi usado a API do ChatGPT (OpenAI) para gerar uma mensagem de clima personalizada para cada cidade.
4. Uma vez que a mensagem para cada cidade esteja pronta, foi criado uma planilha para armazenas a mensagem para cada cidade.



In [ ]:
# Para ter acesso a API OpenWeather, foi seguido o passo a passo do vídeo 'https://www.youtube.com/watch?v=W_AQ_50Njgo&t=1249s'
# Substituir o texto TODO por sua API Key da OpenWeather, ela será salva como uma variável de ambiente.
# Foi alterado o link para requisição da API OpenWeather para puxar informações baseado na Cidade. Também foi configurado para que os valores retornassem em pt-br

openweather_api_key = "KEY"

## **E**xtract

Extraído a lista de Cidades a partir do arquivo CSV. Para cada cidade, foi feito uma requisição GET para obter os dados de clima da cidade correspondente.

In [ ]:
import pandas as pd

df = pd.read_csv('Cidades.csv')
cidades = df['Cidade'].tolist()
print(cidades)

['Manaus', 'Rio de Janeiro', 'São Paulo', 'Salvador', 'Campo Grande']


In [ ]:
import requests
import json
import time

def get_previsao(cidade):
  response = requests.get(f"https://api.openweathermap.org/data/2.5/weather?q={cidade}&appid={openweather_api_key}&lang=pt_br")
  return response.json() if response.status_code == 200 else None

for cidade in cidades:
  print(cidade)
  previsoes = get_previsao(cidade)
  descricao = previsoes['weather'][0]['description']
  print(descricao)
  print(previsoes)
  time.sleep(60)

Manaus
céu limpo
{'coord': {'lon': -60.025, 'lat': -3.1019}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'céu limpo', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 301.42, 'feels_like': 305.6, 'temp_min': 299.43, 'temp_max': 301.42, 'pressure': 1009, 'humidity': 78}, 'visibility': 10000, 'wind': {'speed': 1.03, 'deg': 230}, 'clouds': {'all': 0}, 'dt': 1697865911, 'sys': {'type': 1, 'id': 8412, 'country': 'BR', 'sunrise': 1697881133, 'sunset': 1697925019}, 'timezone': -14400, 'id': 3663517, 'name': 'Manaus', 'cod': 200}
Rio de Janeiro
nublado
{'coord': {'lon': -43.2075, 'lat': -22.9028}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'nublado', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 294.13, 'feels_like': 294.32, 'temp_min': 294.13, 'temp_max': 294.15, 'pressure': 1017, 'humidity': 78}, 'visibility': 10000, 'wind': {'speed': 2.06, 'deg': 160}, 'clouds': {'all': 100}, 'dt': 1697865797, 'sys': {'type': 1, 'id': 8429, 'country': 'BR', 'sunrise': 

## **T**ransform

Utilizado a API do OpenAI GPT-3.5 para gerar uma mensagem sobre o clima personalizada para cidade.

In [ ]:
!pip install openai

In [ ]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = 'KEY'

In [ ]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(descricao, cidade):
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista notícias climáticas."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem sobre o que fazer no clima {descricao} na cidade {cidade} (máximo de 100 caracteres)"
      }
    ]
  )

  return completion.choices[0].message.content.strip('\"')

mensagem =[]

for cidade in cidades:
  news = generate_ai_news(descricao, cidade)
  print(news)
  mensagem.append(news)

## **L**oad

Cria uma planilha de cidade e "news" de cada cidade na API com a nova mensagem gerada.

In [ ]:
dados = dict(zip(cidades, mensagem))

planilha = pd.DataFrame(data=dados, index=[0])
print(planilha)

#planilha.to_excel('mensagem_clima.xls')